In [ ]:
import matplotlib.pyplot as plt

In [1]:
import importlib
import Simulator
importlib.reload(Simulator)
from Simulator import Simulator

neuron_network_filename = "Network3"
simulator = Simulator(neuron_network_filename)
simulator.graph.make_graph_board(0)

#simulator.boarn()

In [3]:
simulator.graph.hierarchical_graph["neuron_model"].node_dict["LIF Trainer/G0/a#0"]

{'label': 'LIF Trainer/G0/a#0',
 'neuron': 'LIF',
 'role': '@Input',
 'locality': 'Local',
 'original_label': 'a#0',
 'location': [[-314.9583435058594, -915.5],
  [0, 0],
  [-522.9583435058594, -711.5]],
 'local_synaps': array([], dtype=float64),
 'local_synaps_index': 0,
 'local_synaps_num': 0,
 'global_synaps': array(['Train/G0/a -> LIF Trainer/G0/a#0 G0'], dtype='<U35'),
 'global_synaps_index': 1,
 'global_synaps_num': 1,
 'depth': 2,
 'hierarchy_name': 'neuron_model'}

In [4]:
simulator.graph.hierarchical_graph["neuron_element"].node_dict["LIF Trainer/G0/a#0/@Input~Global~Spike"]

{'label': 'LIF Trainer/G0/a#0/@Input~Global~Spike',
 'locality': 'Global',
 'role': '@Input',
 'group': 'Else',
 'bound_max': 1,
 'bound_min': 0,
 'Random': 0,
 'initial': 0.1,
 'original_label': 'Spike',
 'location': [[-314.9583435058594, -915.5],
  [0, 0],
  [-522.9583435058594, -711.5],
  [-379.8745935058593, -569.225]],
 'value': {'v': array([0.1]), 'dv': array([0.]), 'ndv': array([0.])},
 'queue': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'time_interval': array([], dtype=float64),
 'depth': 3,
 'hierarchy_name': 'neuron_element'}

In [ ]:
c

In [ ]:
simulator.graph.debug_by_networkx()

In [ ]:
simulator.graph.hierarchical_graph["neuron_element"].node_from_dict

In [ ]:
import random
random.randint(2,5)

In [ ]:
a = {1:'1'}

In [ ]:
list(a.values())

In [ ]:
import math
math.pow(3,2)

In [ ]:
import numpy as np

In [ ]:
a = np.array([])

In [ ]:
a = np.concatenate((a, np.array(['ㅁ'])))
print(a)

In [ ]:
np.where(a==0)[0]

In [ ]:
import time
a = {1:1}
b=0

#a={}


whole_time0 = 0
whole_time1 = 0
whole_time2 = 0

for i in range(1000000) :
    
    start1 = time.time()
    
    for k in a :
        b=b+a[k]
        break
    
    
    end1 = time.time()
    whole_time1 = whole_time1+end1 - start1
    
    
    start2 = time.time()
    
    if len(a)==1 :
        b=b+list(a.values())[0]
    
    
    end2 = time.time()
    whole_time2 = whole_time2+end2 - start2
    
    start0 = time.time()
    end0 = time.time()
    whole_time0 = whole_time0+end0 - start0

print(whole_time1 - whole_time0)
print(whole_time2 - whole_time0)

In [ ]:
neuron_network.graph.hierarchical_graph["neuron_model"].node_dict

In [ ]:
a = {1:'1',2:'2'}
b=a
a[3]='3'
next(iter(a))

In [ ]:
a={}
next(iter(a))

In [ ]:
iter(a)

In [ ]:
a = {1:'1'}
next(iter(a))

In [ ]:
c = dict(a)
c.update(b)
c

In [ ]:
neuron_network.graph.hierarchical_graph["neuron_group"].node_dict

In [ ]:
from random import *
random()

In [ ]:
NeuralNetwork.graph.hierarchical_graph['neuron_model'].node_dict['Inner@1\\n0\\Na+ Channel']["neuron_model"].neuron_node

In [ ]:
import random
import time
import timeit
start = timeit.timeit() 

time.sleep(1)

a = random.random()


print(timeit.timeit() - start)

In [ ]:
import timeit




t1 = timeit.timeit(lambda: "-".join(map(str, range(100))), number=10000)
print(t1)


In [ ]:
NeuralNetwork.network_json

In [ ]:
NeuralNetwork.graph.node_label_dict

In [ ]:
NeuralNetwork.graph.hierarchical_group_dict

In [ ]:
NeuralNetwork.graph.hierarchical_graph['neuron'].edge_dict

In [ ]:
NeuralNetwork.makeInitNeuron()

In [ ]:
NeuralNetwork.neuron_graph.node_dict

In [ ]:
NeuralNetwork.network_json

In [ ]:
NeuralNetwork.neuron_group_dict['Inner@1'].neuron_dict[0].edges

In [ ]:
layer1 = {'nodes':[1,2,3], 'edges':[(1,2),(2,3)]}
layer2 = {'nodes':[1,2,3], 'edges':[(1,2),(1,2)]}
layer3 = {'nodes':[1,2], 'edges':[(1,2)]}
layer4 = {'nodes':[1,2,3,4], 'edges':[(1,2),(1,3),(1,4)]}

In [ ]:
from Graph import Graph

testGraph = Graph()

testNodeB = [1,2,3,4,5,6,7,8]
testEdgeB = [(2,1), (3,1),(4,1),(1,5),(1,6),(1,7), (4,7)]

testNodeA = []
for node in testNodeB :
    testNodeA.append({'label':node})
    
testEdgeA = []
id = 0
for edge in testEdgeB :
    testEdgeA.append({'from':edge[0], 'to':edge[1], 'label':id})
    id = id+1

In [ ]:
testNodeA

In [ ]:
testEdgeA

In [ ]:
testGraphA = {"nodes":testNodeA, "edges":testEdgeA}

In [ ]:
testGraph.initGraph(testGraphA)

In [ ]:
testGraph.deleteNode({'label':1})

In [ ]:
testGraph.node_dict

In [ ]:
testGraph.edge_dict

In [ ]:
testGraph.node_from_dict

In [ ]:
testGraph.node_to_dict

In [ ]:
import networkx as nx

In [ ]:
dd

In [ ]:
a = [[1,2],[3,4]]

In [ ]:
for loc in a : 